In [35]:
import numpy as np

In [1]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header
    return data

In [4]:
train_data = read_data('../dataset/ratings_train.txt')
test_data = read_data('../dataset/ratings_test.txt')

* 데이터가 제대로 읽혔는지 확인

In [5]:
print(len(train_data))
print(len(train_data[0]))

150000
3


In [6]:
print(len(test_data))
print(len(test_data[0]))

50000
3


In [14]:
from konlpy.tag import Twitter
pos_tagger = Twitter()

In [15]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [18]:
train_docs = [(tokenize(row[1].decode('utf-8')), row[2]) for row in train_data[:3000]]
test_docs = [(tokenize(row[1].decode('utf-8')), row[2]) for row in test_data[:1000]]

In [70]:
train_docs[0][1]

<type 'str'>

In [27]:
from pprint import pprint
print(train_docs[0][0][2])

../Punctuation


# doc2vec

In [31]:
train_docs[0][0]

[u'\uc544/Exclamation', u'\ub354\ube59/Noun', u'../Punctuation', u'\uc9c4\uc9dc/Noun', u'\uc9dc\uc99d/Noun', u'\ub098\ub2e4/Verb', u'\ubaa9\uc18c\ub9ac/Noun']

In [41]:
from gensim.models.doc2vec import TaggedDocument

In [42]:
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs]

In [44]:
from gensim.models import doc2vec

In [45]:
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

In [47]:
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=len(train_docs), epochs=epoch)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

In [65]:
for x in doc_vectorizer.most_similar(u'악역/Noun'):
    print(x[0]),
    print(x[1])

배역/Noun 0.942245841026
색다르다/Adjective 0.929360210896
술/Noun 0.909195899963
강하다/Adjective 0.902509450912
애니메이션/Noun 0.892479300499
스릴러/Noun 0.88536298275
감다/Verb 0.879809856415
물론/Noun 0.86969357729
터/Noun 0.868558585644
각본/Noun 0.866173923016
